In [1]:
using Pkg
Pkg.activate("/home/mdoohan/.julia/environments/v1.8/AD_test")

  Activating project at `~/.julia/environments/v1.8/AD_test`


In [2]:
ENV["JULIA_CUDA_VERSION"]="11.4" 
ENV["JULIA_CUDA_USE_BINARYBUILDER"] = true 
using Distributed,CUDA, CMBLensing
@everywhere using CUDA
CUDA.versioninfo()

CUDA toolkit 11.4, artifact installation
NVIDIA driver 535.154.5, for CUDA 12.2
CUDA driver 12.2

Libraries: 
- CUBLAS: 11.6.5
- CURAND: 10.2.5
- CUFFT: 10.5.2
- CUSOLVER: 11.2.0
- CUSPARSE: 11.6.0
- CUPTI: 14.0.0
- NVML: 12.0.0+535.154.5
- CUDNN: 8.30.2 (for CUDA 11.5.0)
- CUTENSOR: 1.4.0 (for CUDA 11.5.0)

Toolchain:
- Julia: 1.8.5
- LLVM: 13.0.1
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0, 7.1, 7.2
- Device capability support: sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80, sm_86

Environment:
- JULIA_CUDA_VERSION: 11.4
- JULIA_CUDA_USE_BINARYBUILDER: true

4 devices:
  0: NVIDIA A100 80GB PCIe (sm_80, 79.148 GiB / 80.000 GiB available)
  1: NVIDIA A100 80GB PCIe (sm_80, 79.148 GiB / 80.000 GiB available)
  2: NVIDIA A100 80GB PCIe (sm_80, 79.148 GiB / 80.000 GiB available)
  3: NVIDIA A100 80GB PCIe (sm_80, 79.148 GiB / 80.000 GiB available)


In [3]:
pathof(CMBLensing)

"/home/mdoohan/.julia/packages/CMBLensing/JIBUG/src/CMBLensing.jl"

In [4]:
@show addprocs(length(CUDA.devices())-1)
CUDA.functional()

addprocs(length(CUDA.devices()) - 1) = [2, 3, 4]


true

In [5]:
@everywhere using CMBLensing, UnPack,  JLD2, Distributions, Random, MuseInference, ComponentArrays, Distributed,  LinearAlgebra, Plots, ForwardDiff, Zygote,  CUDA
                  
@everywhere import MuseInference:  AD, sample_x_z, logLike

In [6]:
pool=Distributed.WorkerPool(procs()) # Should get master GPU working too
CMBLensing.assign_GPU_workers(;use_master=true,remove_oversubscribed_workers=true)

┌ Info: Processes (4):
│  (myid = 1, host = spartan-gpgpu107.hpc.unimelb.edu.au, device = CuDevice(0): NVIDIA A100 80GB PCIe 62a88788))
│  (myid = 2, host = spartan-gpgpu107.hpc.unimelb.edu.au, device = CuDevice(1): NVIDIA A100 80GB PCIe 412262f3))
│  (myid = 3, host = spartan-gpgpu107.hpc.unimelb.edu.au, device = CuDevice(2): NVIDIA A100 80GB PCIe 96c8cd89))
└  (myid = 4, host = spartan-gpgpu107.hpc.unimelb.edu.au, device = CuDevice(3): NVIDIA A100 80GB PCIe a2eca982))


## Make dataset (ds)

In [7]:

 
###### Put fg_ds on Array for cpu's
storage = CuArray
###### rng
rng = MersenneTwister(0)
################## Map/Cl parameters
Cℓ=camb(;ℓmax=7000)
θpix = 2.25#0.42943580147717575# 1.25 
Nside = 256 ; pol = :I ; T = Float64 ; rotator = (0,90,0)
nbins_ϕ = 12 ; log_edges = range(log(100),log(3000), floor(Int,nbins_ϕ+1))
ℓedges_ϕ = T.(exp.(log_edges)) 
ℓedges_T = 200:50:3000 ; nbins_T = length(ℓedges_T)-1
ℓcenters_ϕ = (ℓedges_ϕ[2:end] + ℓedges_ϕ[1:end-1])/2 
ℓcenters_T = (ℓedges_T[2:end] + ℓedges_T[1:end-1])/2 


225.0:50.0:2975.0

In [8]:
@unpack ds,proj = load_sim(;
    # basic configuration
    θpix, Nside, pol, T, storage,
    # noise parameters, or set Cℓn or even Cn directly
    μKarcminT=3.5, ℓknee=1200, αknee=3,
    # beam parameters
    beamFWHM=0,
    # mask parameters
    pixel_mask_kwargs = (;edge_padding_deg = 0.5, edge_rounding_deg = 0.3, apodization_deg = 1, num_ptsrcs = 0),
    bandpass_mask = MidPass(200, 3000; Δℓ=100),
    # theory
    Cℓ, 

    rng,
    L = LenseFlow(10)
    );


In [9]:
Nϕ=ds.Nϕ # QE noise est : Nϕ=quadratic_estimate(ds).Nϕ / Nϕ_fac
nbins_ϕ = length(ℓedges_ϕ)-1
Cϕ=Cℓ_to_Cov(:I, proj,(Cℓ.total.ϕϕ, ℓedges_ϕ, :Aϕ))
G₀ = sqrt(I + Nϕ * pinv(Cϕ()))
Aϕ₀= ones(nbins_ϕ)
G = ParamDependentOp((;Aϕ=Aϕ₀, _...)->(pinv(G₀) * sqrt(I + 2 * Nϕ * pinv(Cϕ(Aϕ=Aϕ)))))


Cf=Cℓ_to_Cov(:I, proj,(Cℓ.unlensed_scalar.TT, ℓedges_T, :AT))
########################## Simulate data

######## Include Cf̃ in dataset
Cf̃  = Cℓ_to_Cov(:I, proj,Cℓ.total.TT)

######
σ²κ = 1e-8
ds = BaseDataSet(;Cf, Cf̃, Cϕ,  Cn=ds.Cn, Cn̂=ds.Cn̂, M=ds.M, M̂=ds.M̂, B=ds.B, B̂=ds.B̂, Nϕ=ds.Nϕ, L=ds.L, D=ds.D, G, σ²κ)
ds.d = simulate(rng,ds).d;
#tru_fields=

LoadError: syntax: incomplete: premature end of input

## Define Muse problem

In [13]:
θ₀ = ComponentVector(;Aϕ=ones(nbins_ϕ), AT=ones(nbins_T))
MAP_joint_kwargs=(;
    nsteps=50, 
    progress=true, 
    aggressive_gc=true, 
    history_keys=(:logpdf), 
    conjgrad_kwargs=(progress=false, tol=1e-1, nsteps=1500),
    prior_deprojection_factor=0)

(nsteps = 50, progress = true, aggressive_gc = true, history_keys = :logpdf, conjgrad_kwargs = (progress = false, tol = 0.1, nsteps = 1500), prior_deprojection_factor = 0)

In [14]:
prob = CMBLensingMuseProblem(ds; MAP_joint_kwargs,
                                autodiff=AD.HigherOrderBackend((AD.ForwardDiffBackend(), AD.ZygoteBackend()))
                                )
∇z_logLike_atol = 1e-2
implicit_diff_cg_kwargs = (maxiter=100, Pl=I)
gc_func = cuda_gc()
𝟘 = zero(θ₀) * zero(θ₀)' # if θ::ComponentArray, helps keep component labels 

z₀ = zero(LenseBasis(FieldTuple(;f=ds.d, ϕ=ds.d)))
T = eltype(z₀)

ad_fwd, ad_rev = AD.second_lowest(prob.autodiff), AD.lowest(prob.autodiff)

(AbstractDifferentiation.ForwardDiffBackend{Nothing}(), AbstractDifferentiation.ReverseRuleConfigBackend{Zygote.ZygoteRuleConfig{Zygote.Context{false}}}(Zygote.ZygoteRuleConfig{Zygote.Context{false}}(Zygote.Context{false}(nothing))))

## Load $\hat{z}$

In [15]:
x=ds.d
z_start = z₀
f = gpu(load_object("fJ_AD_test_SSP1f-7.jld2"))
ϕ = gpu(load_object("ϕJ_AD_test_SSP1f-7.jld2"))
ẑ = LenseBasis(FieldTuple(;f, ϕ));

## AutoDiff parts

### H1

In [16]:
xs=[]
θs=[]
θs_prime=[]
## non-implicit-diff term
H1 =copyto!(similar(𝟘), permutedims(first(AD.jacobian(θ₀, backend=ad_fwd) do θ
    local x, = sample_x_z(prob, copy(rng), θ)
    push!(xs,x)
    push!(θs,θ)
    first(AD.gradient(θ₀, backend=ad_rev) do θ′ 
        #@show θ′
        logLike(prob, x, ẑ, θ′, MuseInference.UnTransformedθ())
    end)
end)))

68×68 ComponentMatrix{Float64} with axes Axis(Aϕ = 1:12, AT = 13:68) × Axis(Aϕ = 1:12, AT = 13:68)
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    

### H2 - part 1

In [17]:
## term involving dzMAP/dθ via implicit-diff (w/ conjugate-gradient linear solve)
dFdθ = first(AD.jacobian(θ₀, backend=ad_fwd) do θ
       # @show θ
    first(AD.gradient(ẑ, backend=ad_rev) do z
        logLike(prob, x, z, θ, MuseInference.UnTransformedθ())
    end)
end)

131072×68 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
  -0.0    -0.0      -0.0       -0.0   …  -0.0069382     0.00670585
  -0.0    -0.0      -0.0       -0.0       0.000934767  -0.00167809
  -0.0    -0.0      -0.0       -0.0       0.00666087   -0.00300012
  -0.0    -0.0      -0.0       -0.0       0.00526456   -0.00386906
  -0.0    -0.0      -0.0       -0.0       0.00284166   -0.00710451
  -0.0    -0.0      -0.0       -0.0   …   0.00224179   -0.000457181
  -0.0    -0.0      -0.0       -0.0       6.49335e-6    0.00815996
  -0.0    -0.0      -0.0       -0.0      -0.00295353    0.0045387
  -0.0    -0.0      -0.0       -0.0      -0.000265485  -0.00210246
  -0.0    -0.0      -0.0       -0.0       0.00538408   -0.00279628
  -0.0    -0.0      -0.0       -0.0   …   0.00402738   -0.00308779
  -0.0    -0.0      -0.0       -0.0      -0.00214236   -0.00160487
  -0.0    -0.0      -0.0       -0.0      -0.00488521    0.00529352
   ⋮                                  ⋱                
 382.521  25.4205  

### H2 - part 2

In [18]:
dFdθ1 = first(AD.jacobian(θ₀, backend=ad_fwd) do θ
    local x, = sample_x_z(prob, copy(rng), θ)
    first(AD.gradient(ẑ, backend=ad_rev) do z
        logLike(prob, x, z, θ₀, MuseInference.UnTransformedθ())
    end)
end)

131072×68 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN 

### H2 
Use parts 1 and 2, and use conjgrad to solve action of inverse hessian

In [56]:

# A is the operation of the Hessian of logLike w.r.t. z
A = MuseInference.LinearMap{T}(length(z₀), isposdef=true, issymmetric=true, ishermitian=true) do w
    first(AD.jacobian(0, backend=ad_fwd) do α
        first(AD.gradient(ẑ + α * w, backend=ad_rev) do z
            logLike(prob, x, z, θ₀, UnTransformedθ())
        end)
    end)
end

cuda_gc()


In [ ]:
A⁻¹_dFdθ1 = pmap(pool, eachcol(dFdθ1)) do w 
    A⁻¹_w = cg(A, w; implicit_diff_cg_kwargs..., log=true)
    cuda_gc()
    A⁻¹_w
end


## H=H1+H2

In [ ]:
cg_hists = map(last, A⁻¹_dFdθ1)
H2 = copyto!(similar(𝟘), -(dFdθ' * mapreduce(first, hcat, A⁻¹_dFdθ1)))

H = H1 + H2